In [ ]:
!pip3 install -r requirements.txt --break-system-packages

In [1]:
import sys
sys.path.insert(0, "~/.local/lib/python3.11/site-packages")

In [3]:
from transformers.utils.import_utils import is_nltk_available
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
%run mongo_query.ipynb

In [7]:
db
hebrew_channels = hebrew_channels

In [18]:
# find hebrew_channel with most messages
hebrew_channel_ids = list(hebrew_channels.keys())
cursor = db["dialogs"].find({"chat.id": {"$in": hebrew_channel_ids}})
top_channels = list(cursor)
# sort
top_channels.sort(key=lambda x: (x["top_message"]["id"]), reverse=True)

In [20]:
top_channels_titles_and_counts = [
    (x["chat"]["id"], x["chat"]["title"], x["top_message"]["id"])
    for x in top_channels
]
print(top_channels_titles_and_counts)

[(-1001162298980, 'רוטר - המדליף', 458584), (-1001001310691, 'החדשות החמות', 333006), (-1001411503185, 'כבאות והצלה ארצי', 270974), (-1001199627330, 'חדשות ישראל קודקוד בטלגרם און ליין ערוץ החדשות של ישראל ® 🆃🅴🅻🅴🅶🆁🅰🅼 @news_kodkodgroup', 155612), (-1001307152557, 'חדשות - הקבינט הביטחוני C.B', 141399), (-1001474443960, 'מבזקי רעם - מבזקי חדשות בזמן אמת', 138864), (-1001985312884, 'Israel News 24/7 - חדשות אמת בטלגרם', 126139), (-1001561068037, 'מבזקי ביטחון 24/7 - ביחד ננצח🇮🇱', 118755), (-1001362537850, '🔴חדשות ישראל בטלגרם🔴', 118413), (-1001254976833, 'קול החדשות ב 🆃🅴🅻🅴🅶🆁🅰️🅼🔴', 103289), (-1001134014032, 'הימין הלא מתנצל של הצל', 96038), (-1001268293343, 'גלובס', 93016), (-1001406113886, 'חדשות מהשטח בטלגרם', 91408), (-1001277927787, 'עמאר אסדי - amar assadi news', 79325), (-1001574173609, '301 העולם הערבי', 74569), (-1001282622805, '⚡️🇮🇱חדשות הבזק🇮🇱⚡️', 72184), (-1001337442223, 'אינטליניוז - חרבות ברזל | חדשות המזרח התיכון | Intellinews', 66138), (-1001143765178, 'אבו עלי אקספרס', 6502

In [23]:
abuali = -1001143765178

In [29]:
db["messages"].find_one(
    {"chat.id": abuali},

)

{'_id': ObjectId('661ca03e2430bc8fb72f0095'),
 'id': 64924,
 'sender_chat': {'id': -1001143765178,
  'type': 'CHANNEL',
  'is_verified': False,
  'is_restricted': False,
  'is_creator': False,
  'is_scam': False,
  'is_fake': False,
  'title': 'אבו עלי אקספרס',
  'username': 'abualiexpress',
  'photo': {'small_file_id': 'AQADBAADtL4xG_zi0FMAEAIAA0Z3LucW____hWSOK-cuD_8ABB4E',
   'small_photo_unique_id': 'AgADtL4xG_zi0FM',
   'big_file_id': 'AQADBAADtL4xG_zi0FMAEAMAA0Z3LucW____hWSOK-cuD_8ABB4E',
   'big_photo_unique_id': 'AgADtL4xG_zi0FM'},
  'dc_id': 4,
  'has_protected_content': False},
 'date': datetime.datetime(2024, 4, 14, 20, 15, 35),
 'chat': {'id': -1001143765178,
  'type': 'CHANNEL',
  'is_verified': False,
  'is_restricted': False,
  'is_creator': False,
  'is_scam': False,
  'is_fake': False,
  'title': 'אבו עלי אקספרס',
  'username': 'abualiexpress',
  'photo': {'small_file_id': 'AQADBAADtL4xG_zi0FMAEAIAA0Z3LucW____hWSOK-cuD_8ABB4E',
   'small_photo_unique_id': 'AgADtL4xG_zi0

In [41]:
result = db["messages"].find_one(
    {
        "chat.id": abuali,
        "$or": [
            {"text": {"$ne": None}},
            {"caption": {"$ne": None}},
            {"poll.question": {"$ne": None}},
        ],
    },
    {
        "from": "$chat.id",
        "date": 1,
        "text": {
            "$ifNull": [
                "$text",
                "$caption",
                "$poll.question",
            ]
        },
        "responses": "$poll.options",
        "reactions": "$reactions.reactions",
    }
)

In [ ]:
model.encode("hello")

In [44]:
result

{'_id': ObjectId('661ca03e2430bc8fb72f0097'),
 'date': datetime.datetime(2024, 4, 14, 20, 15, 34),
 'from': -1001143765178,
 'text': 'שער ברדנבורג, ברלין, גרמניה: הפגנה שנערכה היום לסולידריות עם ישראל אל מול האיום האיראני. בקהל נכחו גולים כורדים וגולים איראנים התומכים בישראל.\nמשתתפי ההפגנה קראו גם להשבת החטופים.\n\nכדי להגיב לכתבה לחצו כאן',
 'reactions': [{'emoji': '❤', 'count': 3604},
  {'emoji': '👍', 'count': 390},
  {'emoji': '🔥', 'count': 54},
  {'emoji': '😁', 'count': 20},
  {'emoji': '💔', 'count': 7},
  {'emoji': '😱', 'count': 3}]}

In [97]:
pipeline = [
    {
        "$match": {
            "chat.id": abuali,
            "$or": [
                {"text": {"$ne": None}},
                {"caption": {"$ne": None}},
            ],
            "reply_to_message_id": None,
        }
    },
    {
        "$project": {
            "cid": "$chat.id",
            "mid": "$id",
            "text": 1,
            "caption": 1,
            "reactions": 1,
            #"views": 1,
            "forwards": 1,
        }
    },
    {
        "$addFields": {
            "reactions_string": {
                "$reduce": {
                    "input": {
                        "$map": {
                            "input": "$reactions.reactions",
                            "as": "reaction",
                            "in": {
                                "$concat": [
                                    "$$reaction.emoji",
                                    ":",
                                    {"$toString": "$$reaction.count"},
                                    ", "
                                ]
                            }
                        }
                    },
                    "initialValue": "",
                    "in": {
                        "$concat": ["$$value", "$$this"]
                    }
                }
            }
        }
    },
    {
        "$addFields": {
            "text_with_reactions": {
                "$concat": [
                    "<<TEXT>>",
                    {"$replaceAll": {
                        "input": {"$ifNull": ["$text", "$caption"]},
                        "find": "\n\nכדי להגיב לכתבה לחצו כאן",
                        "replacement": "",
                    }},
                    "<<REACTIONS>>",
                    {"$rtrim": {"input": "$reactions_string", "chars": ", "}},
                    "<<FORWARDS>>",
                    {"$toString": "$forwards"},
                ]
            }
        }
    },
    {
        "$project": {
            "cid": 1,
            "mid": 1,
            "text": "$text_with_reactions",
        }
    },
    #{"$limit": 100}
]

results = db["messages"].aggregate(pipeline)

for result in results:
    cid, mid, text = result["cid"], result["mid"], result["text"]
    embed = model.encode(text)
    db["embeddings_2"].insert_one({
        "cid": cid,
        "mid": mid,
        "text": text,
        "embedding": embed.tolist(),
    })

# FIXME:
# this is an intermediate solution
# eventually we want:
# - a good encoder that we finetuned on this kind of data
# - a separate encoder for the sentiment (reactions & forwards)
# - and a model to unify them into a single representation
# also, there is no date, and this is terrible!
# date can be multidimensional because different "stories" have different meanings for different dates
# how do we fix this?